In [1]:
import findspark

In [2]:
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.2.1-bin-hadoop3.2'

In [4]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from pyspark.sql import functions

appName = "PySpark PostgreSQL Example - via psycopg2"
master = "local[1]"

spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

In [5]:
url = "jdbc:postgresql://localhost:5432/pagila"
query="'select * from film_list'"
df =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'information_schema.tables')\
    .option("driver", "org.postgresql.Driver") \
.load()


In [6]:
df.show(5)

+-------------+------------+-------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|   table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+-------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|       pagila|      public|   actor_info|      VIEW|                        null|                null|                     null|                    null|                  null|                NO|      NO|         null|
|       pagila|      public|customer_list|      VIEW|                        null|                null|                 

In [8]:
b_tolist=df.rdd.map(lambda x: x['table_name']).collect()
print(b_tolist)

['actor_info', 'customer_list', 'pg_statistic', 'pg_type', 'film_list', 'nicer_but_slower_film_list', 'payment', 'pg_foreign_table', 'pg_authid', 'pg_shadow', 'pg_statistic_ext_data', 'pg_roles', 'pg_settings', 'sales_by_film_category', 'pg_file_settings', 'pg_hba_file_rules', 'pg_config', 'pg_shmem_allocations', 'pg_backend_memory_contexts', 'sales_by_store', 'staff_list', 'pg_available_extension_versions', 'pg_user_mapping', 'pg_stat_xact_user_functions', 'pg_replication_origin_status', 'pg_subscription', 'pg_attribute', 'pg_proc', 'pg_class', 'pg_attrdef', 'pg_constraint', 'pg_inherits', 'pg_index', 'pg_operator', 'pg_opfamily', 'pg_opclass', 'pg_am', 'pg_amop', 'pg_amproc', 'pg_language', 'pg_stat_archiver', 'pg_stat_bgwriter', 'pg_stat_wal', 'pg_stat_progress_analyze', 'pg_stat_progress_vacuum', 'pg_stat_progress_cluster', 'pg_stat_progress_create_index', 'pg_stat_progress_basebackup', 'pg_stat_progress_copy', 'pg_largeobject_metadata', 'pg_aggregate', 'pg_statistic_ext', 'pg_rewr

# вывести количество фильмов в каждой категории, отсортировать по убыванию.

In [9]:
url = "jdbc:postgresql://localhost:5432/pagila"
film_list =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'film_list')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [10]:
film_list.show()

+---+-------------------+--------------------+-----------+-----+------+------+--------------------+
|fid|              title|         description|   category|price|length|rating|              actors|
+---+-------------------+--------------------+-----------+-----+------+------+--------------------+
|730|RIDGEMONT SUBMARINE|A Unbelieveable D...|        New| 0.99|    46| PG-13|JOHNNY LOLLOBRIGI...|
|892|   TITANIC BOONDOCK|A Brilliant Refle...|  Animation| 4.99|   104|     R|BETTE NICHOLSON, ...|
|286|      ENOUGH RAGING|A Astounding Char...|     Travel| 2.99|   158| NC-17|JOHNNY LOLLOBRIGI...|
|857|  STRICTLY SCARFACE|A Touching Reflec...|     Comedy| 2.99|   144| PG-13|GREG CHAPLIN, DAR...|
|593| MONTEREY LABYRINTH|A Awe-Inspiring D...|     Horror| 0.99|   158|     G|KARL BERRY, JULIA...|
|664|      PATRIOT ROMAN|A Taut Saga of a ...|     Action| 2.99|    65|    PG|CARMEN HUNT, ALBE...|
|211|   DARLING BREAKING|A Brilliant Docum...|      Games| 4.99|   165| PG-13|FRANCES DAY-LEWIS...|


In [11]:
film_list.groupBy('category').agg(functions.count('category').alias('quantity')) \
                             .sort(functions.desc('quantity')) \
                             .show()

+-----------+--------+
|   category|quantity|
+-----------+--------+
|    Foreign|      73|
|     Sports|      73|
|     Family|      69|
|Documentary|      68|
|  Animation|      66|
|     Action|      64|
|        New|      63|
|      Drama|      61|
|      Games|      61|
|     Sci-Fi|      61|
|   Children|      60|
|     Comedy|      58|
|   Classics|      57|
|     Travel|      56|
|     Horror|      56|
|      Music|      51|
+-----------+--------+



# вывести 10 актеров, чьи фильмы большего всего арендовали, отсортировать по убыванию

In [12]:
film_actor =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'film_actor')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [13]:
actor =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'actor')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [14]:
film =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'film')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [15]:
actor_film = film.join(film_actor, 'film_id', 'inner').join(actor, 'actor_id', 'inner')

In [16]:
actor_film.groupBy('first_name', 'last_name') \
          .agg(functions.sum('rental_duration').alias('q_days')) \
          .orderBy(functions.desc('q_days')) \
          .show(10) 

+----------+---------+------+
|first_name|last_name|q_days|
+----------+---------+------+
|     SUSAN|    DAVIS|   242|
|      GINA|DEGENERES|   209|
|    WALTER|     TORN|   201|
|      MARY|   KEITEL|   192|
|   MATTHEW|   CARREY|   190|
|    ANGELA|   HUDSON|   183|
|   GROUCHO|    DUNST|   183|
|    SANDRA|   KILMER|   181|
|     HENRY|    BERRY|   180|
|       UMA|     WOOD|   179|
+----------+---------+------+
only showing top 10 rows



# вывести категорию фильмов, на которую потратили больше всего денег

In [17]:
cat_ac_film = film_list.join(film, film_list.fid == film.film_id, 'inner')

In [18]:
from pyspark.sql.functions import col, desc

In [19]:
cat_ac_film.groupBy('category') \
           .agg(functions.sum(col('rental_duration') * col('rental_rate')).alias('total_sum')) \
           .orderBy(desc('total_sum')) \
           .show(1)

+--------+---------+
|category|total_sum|
+--------+---------+
| Foreign|  1179.27|
+--------+---------+
only showing top 1 row



# вывести названия фильмов, которых нет в inventory. Написать запрос без использования оператора IN

In [20]:
inventory =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'inventory')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [21]:
film_inventory = film.join(inventory, 'film_id', 'left')

In [22]:
film_inventory.select('title').filter(col('inventory_id').isNull()).show()

+--------------------+
|               title|
+--------------------+
|      CHOCOLATE DUCK|
|       BUTCH PANTHER|
|        VOLUME HOUSE|
|      ORDER BETRAYED|
|        TADPOLE PARK|
|    KILL BROTHERHOOD|
|FRANKENSTEIN STRA...|
|    CROSSING DIVORCE|
|    SUICIDES SILENCE|
|       CATCH AMISTAD|
|     PERDITION FARGO|
|       FLOATS GARDEN|
|           GUMP DATE|
|        WALLS ARTIST|
|  GLADIATOR WESTWARD|
|         HOCUS FRIDA|
|ARSENIC INDEPENDENCE|
|         MUPPET MILE|
|   FIREHOUSE VIETNAM|
|       ROOF CHAMPION|
+--------------------+
only showing top 20 rows



# вывести топ 3 актеров, которые больше всего появлялись в фильмах в категории “Children”. Если у нескольких актеров одинаковое кол-во фильмов, вывести всех

In [23]:
from pyspark.sql.functions import rank, count
from pyspark.sql.window import Window

In [24]:
list_actor_film = film_list.join(film_actor, film_list.fid == film_actor.film_id, 'inner') \
                           .join(actor, 'actor_id', 'inner')

In [25]:
for_rank = list_actor_film.filter(col('category') == 'Children') \
                          .groupBy('first_name', 'last_name') \
                          .agg(count('fid').alias('q_films'))

In [26]:
WindowSpec = Window.orderBy(desc('q_films'))

In [27]:
for_rank.withColumn("rank",rank().over(WindowSpec)).filter(col('rank') <=3).show()

+----------+---------+-------+----+
|first_name|last_name|q_films|rank|
+----------+---------+-------+----+
|     HELEN|   VOIGHT|      7|   1|
|     SUSAN|    DAVIS|      6|   2|
|     KEVIN|  GARLAND|      5|   3|
|     RALPH|     CRUZ|      5|   3|
|    WHOOPI|     HURT|      5|   3|
|      MARY|    TANDY|      5|   3|
+----------+---------+-------+----+



# вывести города с количеством активных и неактивных клиентов (активный — customer.active = 1). Отсортировать по количеству неактивных клиентов по убыванию

In [28]:
customer =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'customer')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [29]:
customer_list =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'customer_list')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [31]:
customer_city = customer.join(customer_list, customer.customer_id == customer_list.id, 'inner')
one_df = customer_city.filter(col('active') == 1)
zero_df = customer_city.filter(col('active') == 0)

In [32]:
one_df.groupBy('city')\
      .agg(functions.count('active').alias('active'))\
      .join(zero_df.groupBy('city')\
                   .agg(functions.count('active').alias('inactive')), 
            'city', 
            'full')\
      .na\
      .fill(0).orderBy(desc('inactive')).show()

+------------------+------+--------+
|              city|active|inactive|
+------------------+------+--------+
|  Charlotte Amalie|     0|       1|
|            Kamyin|     0|       1|
|            Ktahya|     0|       1|
|           Bat Yam|     0|       1|
|   Southend-on-Sea|     0|       1|
|       Szkesfehrvr|     0|       1|
|          Uluberia|     0|       1|
|           Wroclaw|     0|       1|
|          Xiangfan|     0|       1|
|        Kumbakonam|     0|       1|
|     Coatzacoalcos|     0|       1|
|         Najafabad|     0|       1|
|            Amroha|     0|       1|
|         Pingxiang|     0|       1|
|            Daxian|     0|       1|
|A Corua (La Corua)|     1|       0|
|              Abha|     1|       0|
|         Abu Dhabi|     1|       0|
|             Adana|     1|       0|
|              Acua|     1|       0|
+------------------+------+--------+
only showing top 20 rows



# вывести категорию фильмов, у которой самое большое кол-во часов суммарной аренды в городах (customer.address_id в этом city), и которые начинаются на букву “a”. То же самое сделать для городов в которых есть символ “-”. Написать все в одном запросе.

In [ ]:
inventory =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'inventory')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [ ]:
rental =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'rental')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [ ]:
customer =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'customer')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [ ]:
address =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'address')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [ ]:
city =  spark.read.format("jdbc")\
    .option("url", url)\
    .option("user", 'postgres')\
    .option("password", '52845466')\
    .option("dbtable", 'city')\
    .option("driver", "org.postgresql.Driver") \
.load()

In [ ]:
end_df = film_list.join(film, film_list.fid == film.film_id)\
                  .join(inventory, 'film_id')\
                  .join(rental, 'inventory_id')\
                  .join(customer, 'customer_id')\
                  .join(address, 'address_id')\
                  .join(city, 'city_id')


In [ ]:
end_df.groupBy('city', 'category')\
      .agg(functions.sum('rental_duration').alias('q_hours'))\
      .filter(col('city').startswith('A'))\
      .orderBy(desc('q_hours')).select('category').show(1)

+--------+
|category|
+--------+
|Children|
+--------+
only showing top 1 row



In [ ]:
end_df.groupBy('city', 'category')\
      .agg(functions.sum('rental_duration').alias('q_hours'))\
      .filter(col('city').contains('-'))\
      .orderBy(desc('q_hours')).select('category').show(1)

+--------+
|category|
+--------+
|  Sci-Fi|
+--------+
only showing top 1 row



In [ ]:
!jupyter nbconvert --to python Pyspark_test.ipynb

[NbConvertApp] Converting notebook Pyspark_test.ipynb to python
[NbConvertApp] Writing 7476 bytes to Pyspark_test.py
